<a href="https://colab.research.google.com/github/dmlcks123/minkim1423.github.io/blob/main/ch02_1euichanhwang.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 2장. 통계학습

###1. What is Statistical Learning?

**Y와 X 그리고 *f***

- X는 설명변수(독립변수), Y는 결과변수(종속변수)
<br><br>
- X가 Y와 얼마나 연관이 있는지, 연관이 있다면 얼마나 설명할 수 있는지
  - 연관이 있다면 지도학습을 통해 연관성을 설명하거나(추론) X를 이용하여 Y를 예상(예측)
 <br><br>
- 어떻게 설명할 수 있는지, 설명하는 방법은 검증할 수 있는지
  - 지도학습 중 모수추정법과 비모수추정법을 이용하여 모델을 설정할 수 있음
  - 각각 장단점이 있으며 OLS(ordinary least square)와 SVM(support vector machines)이 있음 
 <br><br>
- 모델로 설명하지 못하는 부분은 어떻게 할것인지
  - 모델로 설명하지 못하는 부분을 오차항(error term)이라고 함 
  - Y와 연관되어 있는 요인들은 무궁무진하기 때문에 관측할 수 있는 X로 Y를 정확히 맞추는 것(fitting)은 불가능
 <br><br>
$$Y = f(X) + \varepsilon$$

<img src = "https://github.com/dmlcks123/minkim1423.github.io/blob/main/img/g1.png?raw=true" width = "400px" height = "300px">
  
 [출처:경제 빅데이터 저장소](https://econbigdata.tistory.com/45)

    - 위의 그림에서 함수 f는 관측치를 이용하여 선형모형을 가정으로한 축소가능한 오차를 최소화하여 추정한 것으로
    - 실제 관측치와 예측치의 차이를 점선으로 나타내었고 이것을 축소가능 오차(reducible error)라고 한다.
<br>

- 모델이 설명하지 못하는 부분은 어떻게 할것인지(irreducible error)
  - 이책의 범위를 넘는다. 어쩔수 없다.
  - X로 설명할 수 있는 error term을 최소화할 수 있는 모델(f)을 추정
  - 모델들을 제작하는 다양한 도구들이 통계적 기법 
<br><br>
- *f*를 추정하는 이유는 예측, 추론을 위해서임



**실습: 데이터 구조 이해 및 X,y 설정하기**

In [ ]:
# Scikit-Learn
# 머신러닝 패키지
# numpy array, pandas dataframe 가능

# 데이터 표현 방식
# X = feature matrix(특징행렬) - 개별관측치(수치값, 이산값, 부울값)
# y = target vector(대상벡터) - 수치값 or 이산클래스/레이블 (numpy array, pandas series)

In [ ]:
#1. numpy library # array

import numpy as np

rs = np.random.RandomState(10) # seed 고정
x = 10 * rs.rand(5) # 난수 생성, 0~10 사이 난수 5개 만듬
y = 2 * x - 1* rs.rand(5)

# x, y의 구조 확인 / 원소가 5개인 1차원 배열
x.shape, y.shape

In [ ]:
# 분석을 위해 열의 형태 데이터로 바꾸기 
X = x.reshape(-1,1) # 열을 1개로 고정시키고 행에 대한 정보를 그대로 가져오겠음 
X.shape # 1차원 --> 2차원, 5행 1열의 데이터로 바뀜

In [ ]:
#2. pandas 실습 # dataframe
# 1차원의 series와 2차원의 dataframe 제공

import seaborn as sns
iris = sns.load_dataset("iris")
iris.info()

In [ ]:
iris.head() # 데이터구조

In [ ]:
# species - rable
# 나머지: feacture_matrix
# --> 구분

# feature matrix(X)
X = iris.drop("species", axis = 1) # df에서 레이블인 Species만 삭제, axis=1은 축 한개 지정한것을 의미
X.shape #150개의 행과 4개의 열로 구성된 feature matrix 완성 ---> X는 튜플 / X의 value들은 np array 임

In [ ]:
y = iris["species"]
y.shape # 1차원 배열 Series

In [ ]:
#3. Bunch 객체를 이용한 X,Y 생성

from sklearn.datasets import load_iris

iris = load_iris()
type(iris)

In [ ]:
iris.keys() # 주요 객체 안내

# naem과 data가 따로 저장되어 있음

iris.data[:5] #[start:end:stop]
# 5개의 행 출력

In [ ]:
iris.data.shape

In [ ]:
iris.target.shape # numpy array 타입

In [ ]:
X = iris.data #특징행렬
y = iris.target #대상행렬(1차원)

**prediction**

- 예측은 관측된 X로 추정한 모형 $\hat{f}$에 X를 대입하여 Y의 예측치를 구하는 과정임

$$\hat{Y} = \hat{f}(X)$$

  - $\hat{Y}$과 $\hat{f}$은 각각 Y의 예측치, *f*의 예측모형을 의미한다.
  - $\hat{Y} = \hat{f}(X)$에서 오차항의 평균은 0이므로 오차항이 소거됨
<br><br>
$$E(Y-\hat{Y})^2 = E[f(X)+\varepsilon -\hat{f}(X)]^2$$
$$               = [f(X)-\hat{f}(X)]^2 + Var(\varepsilon)$$
<br>
  - $E(Y-\hat{Y})^2$은 실제값과 예측값의 차이를 제곱하여 평균을 취한것이고
  - $Var(\varepsilon)$은 오차항의 분산을 의미한다.
    - 여기서 좌변은 제거할수 있는 오차, 우변은 제거할 수 없는 오차를 의미한다.
    - 제거할 수 없는 오차가 존재하는 이유는 Y는 X에 관한 함수이기도 하지만 $\varepsilon$에 대한 함수이기 때문이다.


**Inference**

- 기본 idea
  - 관심있는 X가 Y에 얼마나 영향을 미치는지 추정
  - 설명변수와 반응변수가 관련이 있는지
  - 상관관계와 인과관계를 어떻게 구분할 것인지
  - Y와 X를 선형관계로 가정하고 추정을 할 수 있는지 --> 직관적


**Parametric Methods**

- 모수적 방법은 $f$는 $X$에 대하여 선형적임을 가정
  - $X_{1},...,X_{p}$와 상수항으로만 구성된 선형모델임
  - $f(x)$에 대한 추정을 하는대신 $\beta_{0},...,\beta_{p}$만 추정하면됨
<br><br>
- 훈련데이터를 이용하여 구축한 선형모델로 예측값 $\hat{Y}$가 실제값 $Y$를 잘 예측하도록 훈련
  - 모델의 적합(fit)도가 가장 좋은 모형을 찾는 도구들이 통계적 기법
  - 대표적으로 OLS가 있음
<br><br>
- 장점 : $X$의 계수인 $\beta$와 같은 파라미터들을 추정하므로 $f$를 추정하는 것보다 쉬움
- 단점 : 실제 $f$와 다를 가능성이 큼. 파라미터에 대한 추정이 정확하지 않을 수 있음


**Non-parametric Methods**

- $f$에 대한 가정없이 실제 모형에 적합한 $f$를 찾는 것임
<br><br>
- 장점 : 모수적 추정보다 $\hat{f}$이 실제 $f$와 비슷함
- 단점 : $f$에 대한 추정을 위해 데이터의 사이즈가 커야함 

**Trade-Off**

-  직관적 해석(모수적 추정) vs 예측 정확도(비모수적 추정)
  - Lasso 
  - Least Square
  - Generalized Addictive Models Trees
  - Bagging, Boosting
  - Support Vector Machines
<br><br>
  - 모델로는 선형에서 비선형, 연구방법으로는 해석(ex.정책효과분석)인지 예측(ex.주가예측)인지 결정 

**Supervised vs. Unsupervised Learning**

- 지도학습
  - 모수, 비모수적 추정 모두 해당
  - 설명변수를 측정한 $X_{i}$와 이에 반응하는 반응변수인 $Y_{i}$가 있음
<br><br>
- 비지도학습
  - 측정값 $X_{i}$가 있지만 측정값 $Y_{i}$가 없음
  - 변수, 관측치간의 상관관계 분석
  - ex. 클러스터링 분석
<br><br>
- 준지도학습(semi-supervised learning)
  - 데이터셋에서 $Y_{i}$와 $X_{i}$가 있을 경우 지도학습, $X_{i}$만 있을 경우 비지도학습

**Regression vs. Classification Problem**

- 반응변수(Y)가 양적변수인지 질적변수인지 판단
  - 양적변수일 경우 일반적인 회귀분석 실시(OLS)
  - 질적변수일 경우 $Y$ 분류과정을 거쳐 Logistic regression 실시
    - ex. 통학할때 이용하는 교통수단, 성별
    - ex. 조세저항이 거세지면 정부가 정책을 철회 or 통과시키는지 로짓분석

**예제: 모수추정법-지도학습-양적변수(regression)-선형회귀모델**

In [ ]:
# 데이터 준비
# 패키지 
import numpy as np # numpy형식
import matplotlib.pyplot as plt # plotting

# x,y 설정
# 무작위추출 객체
rs = np.random.RandomState(10) # seed 값 고정(default 0~1 범위)
x = 10 * rs.rand(100)
y = 3 * x + 2 * rs.rand(100) #실제모델

In [ ]:
plt.scatter(x, y, s =10 ); #s=size, 시각화, 선형적인 데이터

In [ ]:
# 위의 모델에 TEST SET을 적용하여 얼마나 에측하는지 확인


# 모델 클래스 선택
# x로 y 예측, 둘다 수치데이터 --> 선형회귀 모델 사용하겠음
# sklearn 패키지에서 linear_model 모듈에서 선형회귀 위의 모델을 사용

# 모델 인스턴스 생성과 하이퍼파라미터 선택

# 인스턴스 생성
from sklearn.linear_model import LinearRegression
regr = LinearRegression() # 인스턴스 만들기 ,생성자 함수 사용

In [ ]:
from sklearn.linear_model import LinearRegression 
regr = LinearRegression(fit_intercept=True) # intercept을 적합시킬것인지 말것인지 설정(옵션)

In [ ]:
# 특징행렬과 대상 벡터 준비

X = x.reshape(-1,1)
X.shape, y.shape #2차원(Numpy 배열), 1차원

In [ ]:
# 모델을 훈련데이터에 적합

regr.fit(X,y) # 모델 인스턴스(선형회귀)를 이용하여 적합 모델을 데이터에 적합

# X를 카피할 것인지, 절편에 적합을 시킬것인지, job의 갯수, 정규화 기능 사용?

In [ ]:
# y = b1X + b0 에서
# b1은 모델에 적합시켰을때 어떻게 나오는지
regr.coef_

In [ ]:
# 상수항 확인
regr.intercept_

In [ ]:
## prediction
# 새로운데이터 생성, 검정 데이터 
x_new = np.linspace(-1, 11, num = 100) # -1에서 11사이에 100개의 선형데이터 생성

In [ ]:
# feature matrix로 전환(2차원)
X_new = x_new.reshape(-1,1)
X_new.shape

In [ ]:
# 모델에 기반하여 y 예측후 저장
y_pred = regr.predict(X_new)

In [ ]:
plt.plot(x_new, y_pred, c = "red") # 예측값(데이터)를 선형그래프로 표시, 색상은 레드
plt.scatter(x, y, s=10); # 실제값

In [ ]:
# 모델 평가
# RMSE (Root mean Squared error) 평균제곱오차
# 실제값과 예측값의 차이

from sklearn.metrics import mean_squared_error #Metrix라고 하느 모듈에서 mse 불러오기

rmse = np.sqrt(mean_squared_error(y, y_pred)) # 루트 씌운 실제값 예측값
rmse
# 0에 가까울수록 best

# 출처: K-ICT 빅데이터센터